# Bayesian AGN Decomposition Analysis for SDSS Spectra - Single Spectrum (Version 7.7.5)

###  Remington O. Sexton (UCR), William Matzko (GMU), Nicholas Darden (UCR), 

some would say its pretty BADASS...

#### Important Warning (READ)

There is a critical defect with scipy version 1.5.1 that will cause an error during the initial fitting process when `scipy.optimize.minimize()` is used:

`ValueError: `x0` violates bound constraints.`

This is an open issue on GitHub ([https://github.com/scipy/scipy/issues/11403](https://github.com/scipy/scipy/issues/11403)) and it is labeled a defect.  As of Septempber 2020, this issue still has not been fixed.  The solution to this problem is to downgrade scipy to version 1.4.1.  If you are using the Anaconda distribution of Python, do the following:

`conda install scipy=1.4.1`

We will update BADASS to fix this problem as soon as the issue is fixed.

In [ ]:
import glob
import time
import natsort
from IPython.display import clear_output
import multiprocess as mp
import os
import psutil
# To see full list of imported packages and modules, see 
# badass3_v7_7_5.py file.
import badass3_v7_7_5 as badass # <<<--- Import BADASS here

### BADASS Options

In [ ]:
################################## Fit Options #################################
# Fitting Parameters
fit_options={
'fit_reg'    : (4400,5500), # Fitting region; Indo-US Library=(3460,9464)
'good_thresh': 0.0, # percentage of "good" pixels required in fig_reg for fit.
'interp_bad' : False, # interpolate over pixels SDSS flagged as 'bad' (careful!)
# Number of consecutive basinhopping thresholds before solution achieved
'n_basinhop': 5,
# Outflow Testing Parameters
'test_outflows': True, 
'outflow_test_niter': 10, # number of monte carlo iterations for outflows
# Maximum Likelihood Fitting for Final Model Parameters
'max_like_niter': 10, # number of maximum likelihood iterations
# LOSVD parameters
'min_sn_losvd': 5,  # minimum S/N threshold for fitting the LOSVD
# Emission line profile parameters
'line_profile':'G' # Gaussian (G) or Lorentzian (L)
}
################################################################################

########################### MCMC algorithm parameters ##########################
mcmc_options={
'mcmc_fit'    : True, # Perform robust fitting using emcee
'nwalkers'    : 100,  # Number of emcee walkers; min = 2 x N_parameters
'auto_stop'   : True, # Automatic stop using autocorrelation analysis
'conv_type'   : 'median', # 'median', 'mean', 'all', or (tuple) of parameters
'min_samp'    : 1500,  # min number of iterations for sampling post-convergence
'ncor_times'  : 5.0,  # number of autocorrelation times for convergence
'autocorr_tol': 10.0,  # percent tolerance between checking autocorr. times
'write_iter'  : 100,   # write/check autocorrelation times interval
'write_thresh': 100,   # when to start writing/checking parameters
'burn_in'     : 7500, # burn-in if max_iter is reached
'min_iter'    : 1500, # min number of iterations before stopping
'max_iter'    : 10000, # max number of MCMC iterations
}
################################################################################

############################ Fit component options #############################
comp_options={
'fit_feii'    : True, # fit broad and narrow FeII emission
'fit_losvd'   : True, # fit LOSVD (stellar kinematics) in final model
'fit_host'    : True, # fit host-galaxy using template (if fit_LOSVD turned off)
'fit_power'   : True, # fit AGN power-law continuum
'fit_broad'   : True, # fit broad lines (Type 1 AGN)
'fit_narrow'  : True, # fit narrow lines
'fit_outflows': True, # fit outflows;
'tie_narrow'  : False,  # tie narrow widths (don't do this)
}
################################################################################

########################### Emission Lines & Options ###########################
# If not specified, defaults to SDSS-QSO Emission Lines (http://classic.sdss.org/dr6/algorithms/linestable.html)
################################################################################

############################### FeII Fit options ###############################
# Below are options for fitting FeII.  For most objects, you don't need to 
# perform detailed fitting on FeII (only fit for amplitudes) use the 
# Veron-Cetty 2004 template ('VC04') (2-6 free parameters)
# However in NLS1 objects, FeII is much stronger, and sometimes more detailed 
# fitting is necessary, use the Kovacevic 2010 template 
# ('K10'; 7 free parameters).

# The options are:
# template   : VC04 (Veron-Cetty 2004) or K10 (Kovacevic 2010)
# amp_const  : constant amplitude (default False)
# fwhm_const : constant fwhm (default True)
# voff_const : constant velocity offset (default True)
# temp_const : constant temp ('K10' only)

feii_options={
'template'  :{'type':'VC04'}, 
'amp_const' :{'bool':False,'br_feii_val':1.0,'na_feii_val':1.0},
'fwhm_const':{'bool':True,'br_feii_val':3000.0,'na_feii_val':500.0},
'voff_const':{'bool':True,'br_feii_val':0.0,'na_feii_val':0.0},
}
# or
# feii_options={
# 'template'  :{'type':'K10'},
# 'amp_const' :{'bool':False,'f_feii_val':1.0,'s_feii_val':1.0,'g_feii_val':1.0,'z_feii_val':1.0},
# 'fwhm_const':{'bool':False,'val':1500.0},
# 'voff_const':{'bool':False,'val':0.0},
# 'temp_const':{'bool':False,'val':10000.0} 
# }
################################################################################

############################# Outflow Test options #############################
# Here one can choose how outflows are fit and tested for 
# Amp. test   : outflow amp. must be N-sigma greater than noise
# FWHM test   : outflow must have greater FWHM than core comp by N-sigma
# VOFF test   : outflow must have a larger offset than core relative to rest;
#               picks out only blueshifted outflows by N-sigma
# Resid. test : there must be a measurable difference in residuals by N-sigma
# Bounds. test: if paramters of fit reach bounds by N-sigma, 
#               consider it a bad fit.
outflow_test_options={
'amp_test':{'test':False,'nsigma':3.0}, # Amplitude-over-noise by n-sigma
'fwhm_test':{'test':False,'nsigma':1.0}, # FWHM difference by n-sigma
'voff_test':{'test':False,'nsigma':1.0}, # blueshift voff from core by n-sigma
'outflow_confidence':{'test':True,'conf':0.95}, # outflow confidence acceptance
'bounds_test':{'test':True,'nsigma':1.0} # within bounds by n-sigma
}
################################################################################

############################### Plotting options ###############################
plot_options={
'plot_param_hist': True,# Plot MCMC histograms and chains for each parameter
'plot_flux_hist' : False,# Plot MCMC hist. and chains for component fluxes
'plot_lum_hist'  : False,# Plot MCMC hist. and chains for component luminosities
'plot_mbh_hist'  : True,# Plot MCMC hist. for estimated AGN lum. and BH masses
'plot_corner'    : False,# Plot corner plot of relevant parameters; Corner plots 
                         # of free paramters can be quite large require a PDF 
                         # output, and have significant time and space overhead, 
                         # so we set this to False by default. 
'plot_bpt'      : True,  # Plot BPT diagram 
}
################################################################################

################################ Output options ################################
output_options={
'write_chain'   : False, # Write MCMC chains for all paramters, fluxes, and
                         # luminosities to a FITS table We set this to false 
                         # because MCMC_chains.FITS file can become very large, 
                         # especially  if you are running multiple objects.  
                         # You only need this if you want to reconstruct chains 
                         # and histograms. 
'print_output'  : True,  # prints steps of fitting process in Jupyter output
}
################################################################################

############################ Multiprocessing options ###########################
# If fitting single object at a time (no for loops!) then one can set threads>1
# If one wants to fit objects sequentially (one after another), it must be set 
# to threads=1, and must use multiprocessing to spawn subprocesses without 
# significant memory leaks. 
mp_options={
'threads' : 4 # number of processes per object
}
################################################################################


### Run BADASS on a single spectrum

The following is shows how to fit single SDSS spectra.

#### Directory Structure

In [ ]:
########################## Directory Structure #################################
spec_dir = 'examples/' # folder with spectra in it
ppxf_dir = 'badass_data_files/' # support files
temp_dir = ppxf_dir+'indo_us_library' # stellar templates
# Get full list of spectrum folders; these will be the working directories
spec_loc = natsort.natsorted( glob.glob(spec_dir+'*') )
################################################################################

#### Choose Spectrum

In [ ]:
nobj = 0 # Object in the spec_loc list
work_dir = spec_loc[nobj]+'/' # working directory
# Set up run ('MCMC_output_#') directory
run_dir,prev_dir = badass.setup_dirs(work_dir,print_output=output_options['print_output']) 
file = glob.glob(work_dir+'*.fits')[0] # Get name of FITS spectra file
#

#### Run 

In [ ]:

# Call the main function in BADASS
badass.run_BADASS(file,run_dir,temp_dir,
                  fit_options,
                  mcmc_options,
                  comp_options,
                  feii_options,
                  outflow_test_options,
                  plot_options,
                  output_options,
                  mp_options,
                 )
    #